<a href="https://colab.research.google.com/github/vggls/msc_thesis_medical_xai/blob/main/experiments/crc_resnet34/HAAS_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Imports**

In [ ]:
pip install grad-cam

In [ ]:
import torch
from torchvision  import datasets, transforms
from pytorch_grad_cam import GradCAM, HiResCAM

In [ ]:
# custom written code
from haas import Haas, HA_image
from densenet121 import _DenseNet121_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Load .zip test data file from Google Drive and unzip it**

In [ ]:
!unzip "./drive/My Drive/Datasets/CRC/CRC-VAL-HE-7K.zip" # test set 7K

# **3. Test dataset**

In [ ]:
# to be applied to validation and test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

In [ ]:
test_path = './CRC-VAL-HE-7K/'
test_dataset = datasets.ImageFolder(root=test_path, transform=test_transforms)

# **4. ResNet34**

In [ ]:
drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/CRC/Models/'
resnet34 = torch.load(drive_path + 'crc_resnet34.pt')
resnet34 = resnet34.cuda()
resnet34 = resnet34.eval()

In [ ]:
resnet34_gradcam_instance = GradCAM(model=resnet34, target_layers=[resnet34.layer4[2].conv2], use_cuda=True)
resnet34_hirescam_instance = HiResCAM(model=resnet34, target_layers=[resnet34.layer4[2].conv2], use_cuda=True)

In [ ]:
print('-->GradCAM')
gradcam_score, _, _, _ = Haas(test_dataset, resnet34, resnet34_gradcam_instance)
print('HAAS score:', gradcam_score)
print('---------------------------')
print('-->HiResCAM')
hirescam_score, _, _, _ = Haas(test_dataset, resnet34, resnet34_hirescam_instance)
print('HAAS score:', hirescam_score)

-->GradCAM
No. of test images:  7180
Total time: 216.14896750450134 secs
Accuracy over original images:  0.91
Accuracy over HA images:  0.43
HAAS score: 0.47
---------------------------
-->HiResCAM
No. of test images:  7180
Total time: 204.56366395950317 secs
Accuracy over original images:  0.91
Accuracy over HA images:  0.48
HAAS score: 0.53
